# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

> You can use the classwork notebook and also [my Everything scraping reference](https://jonathansoma.com/everything/scraping)

## Setup: Import what you'll need to scrape the page

We'll be using either Playwrightfor this, *not* requests.

In [47]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

In [48]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)

In [49]:
page = await browser.new_page()

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [50]:
await page.goto("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")

<Response url='https://www.tdlr.texas.gov/cimsfo/fosearch.asp' request=<Request url='https://www.tdlr.texas.gov/cimsfo/fosearch.asp' method='GET'>>

In [51]:
await page.get_by_label("Search by Last Name").fill("Nguyen")

In [52]:
#await page.get_by_text("Search", exact=True).click()
#await page.get_by_role("button", name="Search").click()
#await page.get_by_role('button[name="Search"]')
search_button = await page.query_selector("input[name='B1']")
if search_button:
    await search_button.click() 

## Scraping

Once you are on the results page, do this. **I step you through things bit by bit, so it's going to be a little different than we did in class.** Also, no `pd.read_html` allowed because this isn't reaaallly tabular data (or there's just too much to clean up).

Once you've loaded the page: **what selector are you going to do use to find the data?** In class we used things like `.article` or `.head-list`.

> You honestly can do this all with Playwright, no BeautifulSoup involved! But we didn't cover that in class so you should stick to this method:
> 
> ```python
> html = await page.content()
> doc = BeautifulSoup(html)
> ```

In [53]:
html = await page.content()
doc = BeautifulSoup(html)

### Grab the rows, loop through each result and print the entire row

It's probably too many rows to do all at once: once you have a list, use `[:10]` to only show the first ten! For example, if you saved the table rows into `results` you might do something like this:

```python
for result in results[:10]:
    print("------")
    print(result)
```

Although you'd want to print out the text from the row (I give example output below).

In [79]:
table = doc.find("tbody").find_all("td")
#rows = table.find("td").find(class_="results_text")

[<td style="padding:4px; text-align:left; font-size:11px; font:Arial, Helvetica, sans-serif; width:22%;"><span class="results_text">NGUYEN, THU THI </span><br/> <span class="default_text">City:</span> <span class="results_text">HOUSTON</span><br/> <span class="default_text">County:</span> <span class="results_text">HARRIS</span><br/> <span class="default_text">Zip Code:</span> <span class="results_text">77031</span><br/><br/><br/><span class="default_text"> License #:</span> <span class="results_text">816854</span><br/><br/><span class="default_text">Complaint #</span> <span class="results_text">COS20230009980</span></td>,
 <td style="padding:4px; text-align:left; font-size:11px; font:Arial, Helvetica, sans-serif; width:39%;"><span class="default_text">Date:</span> <span class="results_text">10/23/2023</span><br/><br/>Respondents Thu Thi Nguyen and Tien Minh Vo are assessed an administrative penalty in the amount of $1,350.</td>,
 <td style="padding:4px; text-align:left; font-size:11px

In [109]:
for i in range(0, len(table), 2):
    spans_row1 = table[i].find_all(['span'], class_=['results_text', 'default_text'])
    spans_row2 = table[i + 1].find_all(['span'], class_=['results_text', 'default_text'])
    
    content_row1 = ''
    content_row2 = ''

    for span in spans_row1:
        text = span.get_text(strip=True)
        if text.startswith("Complaint #"):
            content_row1 += text + ' ' + span.find_next(class_='results_text').get_text(strip=True) + '\n'
        else:
            content_row1 += text + ' '
    for span in spans_row2:
        content_row2 += span.get_text(strip=True) + ' '

    print("Name and Location, Order, Basis for Order")
    print(content_row1.strip().replace('<br/>', '\n').strip())
    print(content_row2.strip())
    print("--------")

Name and Location, Order, Basis for Order
NGUYEN, THU THI City: HOUSTON County: HARRIS Zip Code: 77031 License #: 816854 Complaint # COS20230009980
COS20230009980
Date: 10/23/2023
--------
Name and Location, Order, Basis for Order

NGUYEN, TUONG City: FORT WORTH County: TARRANT Zip Code: 76120 License #: 742271 Complaint # COS20230001960
--------
Name and Location, Order, Basis for Order
Date: 10/4/2023

--------
Name and Location, Order, Basis for Order
NGUYEN, HUONG T City: SAN ANTONIO County: BEXAR Zip Code: 78259 License #: 775126 Complaint # COS20220017901
COS20220017901
Date: 8/23/2023
--------
Name and Location, Order, Basis for Order

NGUYEN, STEVEN QUOC City: SAN ANGELO County: TOM GREEN Zip Code: 76904 License #(s): 748707, 1458935 Complaint # COS20220018135
--------
Name and Location, Order, Basis for Order
Date: 7/28/2023

--------
Name and Location, Order, Basis for Order
NGUYEN, HUONG THI NGOC City: MANSFIELD County: TARRANT Zip Code: 76063 License #: 829081 Complaint # C

IndexError: list index out of range

The result should look something like this:

```
Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, LONG D
City: SAN SABA
County: SAN SABA
Zip Code: 76877
```

### Loop through each result and print each person's name

You'll probably get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything when there's an error, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

Output should look like this:

```
Doesn't have a name
NGUYEN, THANH
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
```

* *Tip: The name has a class you can use. The class name is reused in a lot of places, but because it's the first one you don't have to worry about that!*
* *Tip: Instead of searching across the entire page – like `doc.select_one` – you should be doing your searching just inside of each **row** (I used this technique in the beginning of class with BeautifulSoup when we were scraping the books page)* 

In [106]:
all_results = doc.find_all('span', class_='results_text')
names = [span.get_text(strip=True) for span in all_results if "NGUYEN" in span.get_text(strip=True)]
names

['NGUYEN, THU THI',
 'NGUYEN, TUONG',
 'NGUYEN, HUONG T',
 'NGUYEN, STEVEN QUOC',
 'NGUYEN, HUONG THI NGOC',
 'NGUYEN, TAMMY',
 'NGUYEN, THANG',
 'NGUYEN, TOAN DUC',
 'NGUYEN, NHAN TRONG',
 'NGUYEN, TAM PHUONG',
 'NGUYEN, NHUNG NGOC THI',
 'NGUYEN, ANH NGOC',
 'NGUYEN, TIFFANY',
 'NGUYEN, HUYEN THI',
 'NGUYEN, DANIEL',
 'NGUYEN, BANG',
 'NGUYEN, HONG N',
 'NGUYEN, THANH',
 'NGUYEN, TUYET',
 'NGUYEN, TUYET MINH',
 'NGUYEN, JOHN THANH NHAN',
 'NGUYEN, THE THI',
 'NGUYEN, THAO HONG',
 'NGUYEN, NGOC THI',
 'NGUYEN, TUYEN THANH',
 'NGUYEN, MINH THU',
 'NGUYEN, LOAN THI NGOC',
 'NGUYEN, JACK',
 'NGUYEN, HOANG DRAGON',
 'NGUYEN, KELLY',
 'NGUYEN, NIEN',
 'NGUYEN, AJ',
 'NGUYEN, CHARLIE',
 'NGUYEN, OANH THI THUY',
 'NGUYEN, LOC',
 'NGUYEN, THAI VAN',
 'NGUYEN, THUY THU THI',
 'NGUYEN, DUC VAN',
 'NGUYEN, HUNG THANH',
 'NGUYEN, MIKE NHAN',
 'NGUYEN, SAMANTHA T',
 'NGUYEN, THOMAS',
 'NGUYEN, HIEU VAN',
 'NGUYEN, MYTU THI',
 'NGUYEN, NGOC',
 'NGUYEN, BINH PHUONG THI',
 'NGUYEN, QUI NGOC',
 'NGUYE

## Loop through each result, printing each violation description ("Basis for order")

Your results should look something like:

```
Doesn't have a violation
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
...
```

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: Or I guess you could just skip the one with the problem using try/except...*

## Loop through each result, printing the complaint number

Output should look similar to this:

```
Doesn't have a complaint number
COS20210004784
COS20210009745
COS20210011484
...
```

- *Tip: Think about the order of the elements. Can you count from the opposite direction than you normally do?*

## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

Based on what you print out, the output might look something like:

```
This row is broken: Name and Location Order Basis for Order
{'name': 'NGUYEN, THANH', 'city': 'FRISCO', 'county': 'COLLIN', 'zip_code': '75034', 'complaint_no': 'COS20210004784', 'license_numbers': '790672', 'complaint': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.'}
{'name': 'NGUYEN, LONG D', 'city': 'SAN SABA', 'county': 'SAN SABA', 'zip_code': '76877', 'complaint_no': 'COS20210009745', 'license_numbers': '760420, 1620583', 'complaint': 'Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.'}
```

> _**Tip:** Depending on how you do this, you might want to figure out how to search for "next siblings" or "following siblings"_

### Save that to a CSV named `output.csv`

The dataframe should look something like...

|index|name|city|county|zip_code|complaint_no|license_numbers|complaint|
|---|---|---|---|---|---|---|---|
|0|NGUYEN, THANH|FRISCO|COLLIN|75034|COS20210004784|790672|Respondent failed to clean and sanitize whirlp...|
|1|NGUYEN, LONG D|SAN SABA|SAN SABA|76877|COS20210009745|760420, 1620583|Respondent failed to keep a record of the date...|


- *Tip: If you send a list of dictionaries to `pd.DataFrame(...)`, it will create a dataframe out of that list!*

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.